In [1]:
import pandas as pd
import yaml
import sys
import os
from glob import glob
from pathlib import Path
import pickle

import warnings
warnings.filterwarnings('ignore')

sys.path.append('/kaggle/src')
from utils.xgb import fit_xgb
from utils.metric import compute_comptetition_metric
from utils.postprocess import post_process
from utils.set_seed import seed_base
from feature_engineering.stage2 import generate_2nd_stage_features

PACKAGE_DIR = Path("/kaggle/src")
CFG = yaml.safe_load(open(PACKAGE_DIR / "config.yaml", "r"))
print(CFG["2nd_stage"]["execution"]["exp_id"])

CFG["output_dir"] = f"/kaggle/output/{CFG['2nd_stage']['execution']['exp_id']}"
!rm -r {CFG["output_dir"]}
os.makedirs(CFG["output_dir"], exist_ok=True)

seed_base(CFG["env"]["seed"])

2023-10-01 16:25:00.003501: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-01 16:25:00.239423: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-01 16:25:01.375584: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/u

exp_051
rm: cannot remove '/kaggle/output/exp_051': No such file or directory


In [2]:
# 特徴生成
files = glob(f"{CFG['dataset']['step_csv_dir']}/*.csv")
train, features = generate_2nd_stage_features(files)

# cv splitとマージ
cv_split = pd.read_csv(CFG['dataset']['cv_split_path'])
train["fold"] = train["series_id"].map(cv_split.set_index("series_id")["fold"])
display(train.head(5))
train.shape

generate features: 100%|██████████| 277/277 [07:44<00:00,  1.68s/it]


,anglez_diff_abs,anglez_diff_abs_rolling_max_10,anglez_diff_abs_rolling_max_100,anglez_diff_abs_rolling_max_1000,anglez_diff_abs_rolling_max_50,anglez_diff_abs_rolling_mean_10,anglez_diff_abs_rolling_mean_100,anglez_diff_abs_rolling_mean_1000,anglez_diff_abs_rolling_mean_50,anglez_diff_abs_rolling_median_10,...,anglez_mean_rolling_median_1000_shift_434,enmo_mean_rolling_median_1000_shift_434,anglez_rolling_square_mean_1000_shift_434,enmo_rolling_square_mean_1000_shift_434,anglez_diff_abs_rolling_square_mean_1000_shift_434,enmo_diff_abs_rolling_square_mean_1000_shift_434,anglez_mean_rolling_square_mean_1000_shift_434,enmo_mean_rolling_square_mean_1000_shift_434,reduce_step,fold
296,11.684900,36.994174,51.115900,84.6226,51.115900,11.261071,12.183105,7.862673,12.517113,8.006350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6819,3
297,13.075613,30.996570,51.115900,84.6226,44.529509,13.301110,12.761695,7.463610,12.094577,12.030083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6842,3
298,12.413470,33.371791,45.949874,84.6226,39.665117,12.084873,11.926955,7.082522,12.905041,8.490039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6865,3
299,13.615730,30.516778,43.971200,84.6226,37.958600,12.941608,13.244629,6.773975,11.794722,11.172298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6888,3
300,9.200848,28.272274,49.273557,84.6226,43.971200,10.714105,13.011863,6.424168,13.524268,8.659978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6911,3


(906229, 1380)

In [3]:
# 学習
trn_oof, models = fit_xgb(
    X=train, 
    y=train["target"], 
    folds=train["fold"].astype(int), 
    features=features.all_features(),
    params=CFG["2nd_stage"]["xgboost"], 
    es_rounds=100,
)
train["oof"] = trn_oof
train = train.drop(columns=list(set(features.all_features()) - set(["minutes"])))
train = train.sort_values(["series_id", "step"]).reset_index(drop=True)

# nanは1st stageの結果を使う
cand_path = os.path.join("/kaggle/output", CFG["1st_stage"]["execution"]["best_exp_id"], "oof.csv")
last_train = pd.read_csv(cand_path).rename(columns={"oof": "oof_1st"})
train = last_train[["series_id", "oof_1st", "step"]].merge(
    train[["series_id", "oof", "step", "minutes"]], 
    on=["series_id", "step"], 
    how="left")
train["oof"] = train["oof"].fillna(train["oof_1st"])
train = train.sort_values(["series_id", "step"]).reset_index(drop=True)

# 保存
for i, model in enumerate(models):
    model.save_model(os.path.join(CFG["output_dir"], f'xgb_fold{i}.model'))
with open(os.path.join(CFG["output_dir"], "features.pkl"), "wb") as f:
    pickle.dump(features, f)
train.to_csv(os.path.join(CFG["output_dir"], "oof.csv"), index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

== fold 0 ==
[0]	eval-logloss:0.64444
[100]	eval-logloss:0.26741
[200]	eval-logloss:0.26579
[289]	eval-logloss:0.26751


 20%|██        | 1/5 [00:36<02:26, 36.62s/it]

== fold 1 ==
[0]	eval-logloss:0.64618
[100]	eval-logloss:0.24401
[200]	eval-logloss:0.23945
[300]	eval-logloss:0.23975
[314]	eval-logloss:0.23998


 40%|████      | 2/5 [01:17<01:57, 39.03s/it]

...

 60%|██████    | 3/5 [02:05<01:26, 43.00s/it]

== fold 3 ==
[0]	eval-logloss:0.64755
[100]	eval-logloss:0.26093
[200]	eval-logloss:0.25628
[300]	eval-logloss:0.25588
[358]	eval-logloss:0.25626


 80%|████████  | 4/5 [02:45<00:42, 42.06s/it]

== fold 4 ==
[0]	eval-logloss:0.64632
[100]	eval-logloss:0.30694
[166]	eval-logloss:0.30544


100%|██████████| 5/5 [03:10<00:00, 38.16s/it]


In [4]:
# 後処理
sub = post_process(train)
sub = sub.merge(train[["series_id", "step", "minutes"]].rename(columns={"step": "key_step"}), on=["series_id", "key_step"], how="left")
sub.to_csv(os.path.join(CFG["output_dir"], "submission.csv"), index=False)

# スコア計算
labels = pd.read_csv(f"{CFG['dataset']['competition_dir']}/train_events.csv").dropna()
score, ap_table = compute_comptetition_metric(labels, sub)
print(f"score: {score:.4f}")
display(ap_table)

post process: 100%|██████████| 277/277 [00:48<00:00,  5.73it/s]


score: 0.6321


event   tolerance
onset   12           0.027006
        36           0.255948
        60           0.475076
        90           0.620108
        120          0.705362
        150          0.757580
        180          0.786737
        240          0.816764
        300          0.841402
        360          0.851437
wakeup  12           0.084483
        36           0.363847
        60           0.534821
        90           0.656585
        120          0.720839
        150          0.766094
        180          0.798265
        240          0.836974
        300          0.863222
        360          0.879370
dtype: float64